# Bader Charge

For the source code, see [bader](https://workgraph-collections.readthedocs.io/en/latest/qe/module.html#workgraph_collections.ase.espresso.bader.bader_workgraph).


## Visualizing the WorkGraph Builder


In [1]:
from workgraph_collections.qe.bader import bader_workgraph
from aiida import load_profile
load_profile()

node = bader_workgraph.node()
node.to_html()

## Visualizing the WorkGraph

In [2]:
from workgraph_collections.qe.bader import bader_workgraph

wg = bader_workgraph()
wg.to_html()

/home/xing/miniconda3/envs/aiida/lib/python3.11/site-packages/aiida/engine/processes/ports.py:148: UserWarning: default of input port `charge_density_filename` is a `Node` instance, which can lead to unexpected side effects. It is advised to use a lambda instead, e.g.: `default=lambda: orm.Int(5)`.
  warnings.warn(UserWarning(message))
/home/xing/miniconda3/envs/aiida/lib/python3.11/site-packages/aiida/engine/processes/ports.py:148: UserWarning: default of input port `reference_charge_density_filename` is a `Node` instance, which can lead to unexpected side effects. It is advised to use a lambda instead, e.g.: `default=lambda: orm.Int(5)`.
  warnings.warn(UserWarning(message))


## Example: H2O molecule

### Prepare the inputs and submit the workflow


In [8]:
from aiida import load_profile
from aiida.orm import Dict, KpointsData, StructureData, load_code
from aiida_pseudo.data.pseudo import UpfData
from ase.build import molecule
from workgraph_collections.qe.bader import bader_workgraph


load_profile()
#===============================================================================
# load the codes
pw_code = load_code("qe-7.2-pw@localhost")
pp_code = load_code("qe-7.2-pp@localhost")
bader_code = load_code("bader@localhost")
# ===============================================================================
# create input structure node
atoms = molecule("H2O")
atoms.center(vacuum=3.0)
atoms.pbc = True
structure = StructureData(ase=atoms)
# create input parameters node
scf_paras = Dict(
    {
        "CONTROL": {
            "calculation": "scf",
        },
        "SYSTEM": {
            "ecutwfc": 30,
            "ecutrho": 240,
            "occupations": "smearing",
            "smearing": "gaussian",
            "degauss": 0.1,
        },
    }
)
kpoints = KpointsData()
kpoints.set_kpoints_mesh([1, 1, 1])
# Load the pseudopotential kjpaw.
pseudos = {"H": UpfData("/home/xing/data/ase/espresso_pseudo/kjpaw/H.pbe-kjpaw_psl.1.0.0.UPF"),
           "O": UpfData("/home/xing/data/ase/espresso_pseudo/kjpaw/O.pbe-n-kjpaw_psl.1.0.0.UPF")}
#
#
metadata = {
    "options": {
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        },
    }
}
# ===============================================================================
inputs = {"scf":{
                "pw": {
                    "parameters": scf_paras,
                    "pseudos": pseudos,
                    "metadata": metadata
                },
                "kpoints": kpoints,
            },

        "pp_valence": {
                "parameters": Dict({
                        "INPUTPP": {"plot_num": 0},
                        "PLOT": {"iflag": 3},
                    }),
                "metadata": metadata,
        },
        "pp_all": {
                "parameters": Dict({
                        "INPUTPP": {"plot_num": 21},
                        "PLOT": {"iflag": 3},
                    }),
                "metadata": metadata,
        },
        "bader": {
            "metadata": metadata,
        }
}
# prepare inputs and submit
wg = bader_workgraph(structure=structure, pw_code=pw_code,
                     pp_code=pp_code, bader_code=bader_code,
                     inputs=inputs)
wg.submit(wait=True, timeout=300)
#------------------------- Print the output -------------------------
charges = wg.tasks["bader"].node.outputs.bader_charge.get_array("charge")
print("Bader charges:")
print("Index    Symbol    Charge")
for i, charge in enumerate(charges):
    print(f"{i:5d}    {atoms.get_chemical_symbols()[i]:5s}    {charge:5.3f}")


WorkGraph node created, PK: 53980
Bader charges:
Index    Symbol    Charge
    0    O        7.147
    1    H        0.427
    2    H        0.427
